In [233]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [234]:
def get_data(filename):
    data = pd.read_csv(filename, names=['Time', 'Voltage'], dtype={"Time": "string", "Voltage": "string"})
    data = data[2:]
    data['Time'] = pd.to_numeric(data['Time'])
    data['Voltage'] = pd.to_numeric(data['Voltage'])
    data['Time'] += abs(data['Time'].iloc[0])
    
    return data

In [302]:
data1 = get_data('data_raw/C/scope_11_1.csv') # low
data2 = get_data('data_raw/C/scope_11_2.csv') # high

In [303]:
# oscilloscope applied a factor of 10 to high signal, unsure why
data2['Voltage'] *= 10

In [304]:
voltage_diff = data2['Voltage'] - data1['Voltage']
data_diff = pd.DataFrame({'Time': data1['Time'], 'Voltage': voltage_diff})

In [336]:
data_diff_trim = data_diff

In [349]:
# plt.figure(figsize=(10, 6))
# plt.plot(data_diff_trim['Time'], data_diff_trim['Voltage'], marker='', linestyle='-', label='Differential')
# plt.plot(data1['Time'][:2000], data1['Voltage'][:2000], marker='', linestyle='-', label='CANL')
# plt.plot(data2['Time'][:2000], data2['Voltage'][:2000], marker='', linestyle='-', label='CANH')
# plt.title('CAN Signal Sample')
# plt.xlabel('Time (s)')
# plt.ylabel('Voltage (V)')
# plt.grid(True)
# plt.legend()
# plt.show()

In [338]:
voltage = np.array(data_diff_trim['Voltage'])
time = np.array(data_diff_trim['Time'])

In [339]:
# voltage thresholds (paper)
#   rising edge = 0.85 V
#   falling edge = 0.2 V

In [348]:
# convert to square wave
rising_thresh = 0.85
falling_thresh = 0.2

if voltage[0] > 1:
    current_state = 1 # high
else:
    current_state = 0 # low

voltage_square = []

for v in voltage:
    if current_state == 0 and v < rising_thresh:
        voltage_square.append(0)
    elif current_state == 0 and v >= rising_thresh:
        voltage_square.append(1)
        current_state = 1
    elif current_state == 1 and v >= falling_thresh:
        voltage_square.append(1)
    elif current_state == 1 and v < falling_thresh:
        voltage_square.append(0)
        current_state = 0
        

# plt.figure(figsize=(10, 2))
# # plt.plot(data_diff_trim['Time'], data_diff_trim['Voltage'], marker='', linestyle='-', label='Differential')
# plt.plot(time, voltage_square, marker='', linestyle='-', label='Binary Signal')
# plt.title('CAN Signal Sample')
# plt.xlabel('Time (s)')
# plt.ylabel('Signal Value')
# plt.grid(True)
# plt.legend()
# plt.show()

In [341]:
# get indices of bit changes
transition_idx = np.where(np.diff(voltage_square) != 0)[0] + 1

# get time of bit changes
transition_time = time[transition_idx]

# get length of constant bit values
transition_time_diff = np.diff(transition_time)

# split into 0 (dominate) and 1 (recessive) bits
if voltage_square[0] == 0:
    transition_time_diff_1 = transition_time_diff[::2]
    transition_time_diff_0 = transition_time_diff[1::2]
else:
    transition_time_diff_0 = transition_time_diff[::2]
    transition_time_diff_1 = transition_time_diff[1::2]

In [244]:
def prepare_bit_times(transition_time_diff):
    # remove inter-frame gap
    # at most 5 consecutive bits = .000004*5 = .00002
    threshold = 0.000022
    indices = np.where(transition_time_diff > threshold)[0]
    transition_time_diff_trim = list(np.delete(transition_time_diff, indices))
    
    # split repeat bits
    idx = 0
    while (idx < len(transition_time_diff_trim)):
        val = transition_time_diff_trim[idx]
        
        # 1 bit
        # .000004
        if (val < 0.000005):
            idx += 1
            continue
            
        # two bits
        # .000008
        elif (val > 0.000005) and (val < 0.000010):
            new_vals = [val/2, val/2]
            transition_time_diff_trim = transition_time_diff_trim[:idx] + new_vals + transition_time_diff_trim[idx+1:]

            idx += 2
        
        # three bits
        # .000012
        elif (val > 0.000010) and (val < 0.000013):
            new_vals = [val/3, val/3, val/3]
            transition_time_diff_trim = transition_time_diff_trim[:idx] + new_vals + transition_time_diff_trim[idx+1:]

            idx += 3
        
        # four bits
        # .000016
        elif (val > 0.000013) and (val < 0.000018):
            new_vals = [val/4, val/4, val/4, val/4]
            transition_time_diff_trim = transition_time_diff_trim[:idx] + new_vals + transition_time_diff_trim[idx+1:]

            idx += 4
        
        # five bits
        # .000016
        elif (val > 0.000018) and (val < 0.000022):
            new_vals = [val/5, val/5, val/5, val/5, val/5]
            transition_time_diff_trim = transition_time_diff_trim[:idx] + new_vals + transition_time_diff_trim[idx+1:]

            idx += 5
               
        else:
            print(val)
            print("Error")
            
    return transition_time_diff_trim

In [245]:
bit_times_0 = prepare_bit_times(transition_time_diff_0)
bit_times_1 = prepare_bit_times(transition_time_diff_1)

In [246]:
print(min(bit_times_0), max(bit_times_0))
print(min(bit_times_1), max(bit_times_1))

3.942399900001836e-06 4.019200100000009e-06
3.968000000000027e-06 4.0448001000018e-06


In [247]:
# save data
np.savetxt('data_raw_2/C0_2.txt', bit_times_0)
np.savetxt('data_raw_2/C1_2.txt', bit_times_1)

In [268]:
first = np.loadtxt('data_raw_2/C1_1.txt')
second = np.loadtxt('data_raw_2/C1_2.txt')

concat = np.concatenate((first, second))
np.savetxt('data_raw_2/C1.txt',concat)

In [294]:
data0 = np.loadtxt('data_raw_2/C0.txt')
data1 = np.loadtxt('data_raw_2/C1.txt')

In [346]:
def extract_features(data, bits_per_data_pt):
    i = 0
    j = bits_per_data_pt
    
    data_features = []
    
    while j < data.shape[0]:
        data_slice = data[i:j]

        mean = np.mean(data_slice)
        std = np.std(data_slice)
        var = np.var(data_slice)
        skew = stats.skew(data_slice)
        kurtosis = stats.kurtosis(data_slice)
        rms = np.sqrt(np.mean(data_slice**2))
        # high = np.max(data_slice)
        energy = np.mean(data_slice**2)

        i += bits_per_data_pt
        j += bits_per_data_pt
        
        data_features.append([mean, std, var, skew, kurtosis, rms, energy])
    
    return data_features

In [347]:
bits_per_data_pt0 = (data0.shape[0]//140)*5
bits_per_data_pt1 = (data1.shape[0]//140)*5


data0_features = np.array(extract_features(data0, bits_per_data_pt0))
data1_features = np.array(extract_features(data1, bits_per_data_pt1))

In [297]:
data_features = np.concatenate((data0_features, data1_features), axis=1)

In [298]:
np.savetxt('data_2/C_features.txt', data_features)

In [254]:
# dataA = np.loadtxt('data/features/A_features.txt')
# dataB = np.loadtxt('data/features/B_features.txt')
# dataC = np.loadtxt('data/features/B_features.txt')

In [255]:
# targetA = np.full(dataA.shape[0], 0)
# targetB = np.full(dataB.shape[0], 1)
# targetC = np.full(dataC.shape[0], 2)

In [256]:
# all_data = np.concatenate((dataA, dataB, dataC), axis=0)
# all_target = np.concatenate((targetA, targetB, targetC), axis=0)

In [257]:
# all_target

In [258]:
# 15730 bits
# 112 bits / message
# 140 frames
# 5 frames per data point
# 28 data points

In [259]:
# bit time approx
# 500*10^3 bits per second
# 2*10^6 seconds per bit
# experimental bit rate is 4*10^6 seconds per bit

In [260]:
# 645,000 sps

In [261]:
# max data points is 2,000,000 (2 Mhz)
# we want 50,000,000 sps
# 0.04 sec to get same sampleing rate

# 1 frame is 441*10^-6 s
# 0.04 sec / 441*10^-6 s ~= 90 frames

# bitrate = 500*10^3 bps
# 1 frame ~80 bits
# 1 frame = 0.00016 seconds = 160 * 10^-6 s

# timeout > .5 ms = 1 ms

# 0.04/10 = 0.004 = 4ms

# 45 frames / 5 = 9 data points

# 1 = LOW
# 2 = HIGH
# diff = 1-2